# Data preparation for regression analysis

## 1. Import packages

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

## 2. Load data and create individual data frames

### 2.1 2-week repo rate

In [2]:
repo_rate = pd.read_csv("../../data/cnb_repo.txt", sep="|")

In [3]:
repo_rate["VALID_FROM"] = pd.to_datetime(repo_rate["VALID_FROM"], format="%Y%m%d")

In [4]:
repo_rate = repo_rate.rename(columns={"VALID_FROM": "date", "CNB_REPO_RATE_IN_%": "r"})

In [5]:
repo_rate = repo_rate[repo_rate["date"] >= "1998-01-22"]

In [6]:
repo_rate["delta_r"] = repo_rate["r"].diff()

In [7]:
repo_rate.head()

,date,r,delta_r
31,1998-03-20,15.0,NaN
32,1998-07-17,14.5,-0.5
33,1998-08-14,14.0,-0.5
34,1998-09-25,13.5,-0.5
35,1998-10-27,12.5,-1.0


### Sentiments

In [8]:
sentiments = pd.read_csv(
    "../../predictions/sentiment_predictions.tsv", sep="\t", header=0, names=["date", "sentiment", "hawk_pref_score"]
)

In [9]:
dates = sentiments["date"].str.split(".")

In [10]:
sentiments["date"] = dates.apply(lambda x: x[0])

In [11]:
sentiments["date"] = pd.to_datetime(sentiments["date"], format="%Y%m%d")

In [12]:
sentiments.head()

,date,sentiment,hawk_pref_score
0,1998-01-22,neutral,-0.661030
1,1998-02-19,neutral,0.540458
2,1998-03-19,neutral,1.018434
3,1998-04-30,hawkish,0.194856
4,1998-05-28,neutral,0.082861


### Voting records

Here, I am computing disagreement as the standard deviation of suggested policy rate changes among board members.

In [13]:
votings = pd.read_excel("../../data/voting_of_the_bank_board.xlsx", sheet_name="1998-2024", header=None)

In [14]:
votings = votings.T

In [15]:
votings[0] = votings[0].fillna(method="ffill")

/var/folders/n4/k1wgly0j7nx3d42shbk1b3bh0000gn/T/ipykernel_6732/1322423382.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  votings[0] = votings[0].fillna(method="ffill")
/var/folders/n4/k1wgly0j7nx3d42shbk1b3bh0000gn/T/ipykernel_6732/1322423382.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  votings[0] = votings[0].fillna(method="ffill")


In [16]:
votings.columns = ["year", "day_month"] + votings.iloc[0, 2:].to_list()

In [17]:
votings = votings.drop(0, axis=0)

In [18]:
votings["year"] = votings["year"].astype(int).astype(str)

In [19]:
votings["date"] = votings["day_month"].str.strip() + votings["year"].str.strip()

In [20]:
votings["date"] = pd.to_datetime(votings["date"], format="%d.%m.%Y")

In [21]:
votings = votings.drop(["year", "day_month"], axis=1)

In [22]:
votings = votings.replace("o", np.nan).replace(" ", np.nan)

/var/folders/n4/k1wgly0j7nx3d42shbk1b3bh0000gn/T/ipykernel_6732/1463594509.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  votings = votings.replace("o", np.nan).replace(" ", np.nan)


In [23]:
votings["disagreement"] = votings.drop("date", axis=1).var(axis=1)

In [24]:
votings["disagreement"] = votings["disagreement"].replace(np.nan, 0)

In [25]:
votings = votings.drop(columns=[col for col in votings.columns if col not in ["date", "disagreement"]])

In [26]:
votings = votings[votings["date"] <= "2024-09-25"]

In [27]:
votings.head()

,date,disagreement
1,1998-01-22,0.000
2,1998-02-19,0.000
3,1998-03-19,0.075
4,1998-04-30,0.000
5,1998-05-28,0.000


### Inflation rate

HICP for the CZ from the ECB: https://tinyurl.com/bdyn72sk [retrieved: 19.10.2024]

In [28]:
inflation = pd.read_csv(
    "../../data/hicp_cz_monthly.csv", header=0, usecols=["DATE", "HICP - Overall index (ICP.M.CZ.N.000000.4.ANR)"]
)

In [29]:
inflation.columns = ["date", "hicp"]

In [30]:
inflation = inflation[inflation["date"] >= "1998-01-01"]

In [31]:
inflation["date"] = pd.to_datetime(inflation["date"])

In [32]:
inflation.head()

,date,hicp
24,1998-01-31,12.1
25,1998-02-28,12.4
26,1998-03-31,12.4
27,1998-04-30,12.0
28,1998-05-31,12.0


## 3. Merge dataframes and export finished dataframe as csv

In [33]:
def bin(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0

In [34]:
sentiment_votings = pd.merge(sentiments, votings, how="left", on="date")

In [35]:
sentiment_votings = sentiment_votings.sort_values("date")

In [36]:
repo_rate = repo_rate.sort_values("date")

In [37]:
# Possibly change to direction="forward"!!
svrr = pd.merge_asof(
    sentiment_votings, 
    repo_rate, 
    on="date", 
    direction="forward",
    tolerance=pd.Timedelta(days=7)
)

In [38]:
svrr = svrr.fillna(0)

In [39]:
svrr["delta_r_cat"] = svrr["delta_r"].apply(lambda x: bin(x))

In [40]:
data = pd.merge_asof(
    svrr,
    inflation,
    on="date",
    direction="nearest",
    tolerance=pd.Timedelta(days=16)
)

In [41]:
data

,date,sentiment,hawk_pref_score,disagreement,r,delta_r,delta_r_cat,hicp
0,1998-01-22,neutral,-0.661030,0.000000,0.00,0.00,0,12.1
1,1998-02-19,neutral,0.540458,0.000000,0.00,0.00,0,12.4
2,1998-03-19,neutral,1.018434,0.075000,15.00,0.00,0,12.4
3,1998-04-30,hawkish,0.194856,0.000000,0.00,0.00,0,12.0
4,1998-05-28,neutral,0.082861,0.000000,0.00,0.00,0,12.0
...,...,...,...,...,...,...,...,...
260,2024-03-20,neutral,-0.639292,0.014881,5.75,-0.50,-1,2.2
261,2024-05-02,neutral,0.767536,0.000000,5.25,-0.50,-1,3.1
262,2024-06-27,dovish,-0.091087,0.014881,4.75,-0.50,-1,2.2
263,2024-08-01,dovish,0.399827,0.000000,4.50,-0.25,-1,2.5


In [42]:
data.to_csv("../../data/reg_data_cz.csv", index=False)